In [1]:
!pip3 install matplotlib numpy pylzma ipykernel

******************START OF CODE******************

Bigram model and playing around ::

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#print(device)

block_size = 8
batch_size = 4
max_iters = 1000
learning_rate = 3e-4
eval_iters = 250
# dropout = 0.2

In [3]:
with open("wizard of oz for llm.txt", 'r', encoding='utf-8') as f: #open smaple training data file as read with utf 8 encoding
  text = f.read()
chars = sorted(set(text)) #creates a sorted list of all used characters
print(chars)
vocabulary_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [4]:
string_to_int = { ch:i for i,ch in enumerate(chars) } #mappings for str to int and vice versa
int_to_string = { i:ch for i,ch in enumerate(chars) } # above
encode = lambda s: [string_to_int[c] for c in s] #encoder
decode = lambda l: ''.join([int_to_string[i] for i in l]) #decoder

#example NOT actual code, comment out when not testing/demo-ing =>

'''encoded_hello = encode('hello') #encode string 'hello' to integars
decoded_hello = decode(encoded_hello)
print(encoded_hello)
print(decoded_hello)'''

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([79, 32, 72,  5, 71,  1, 66, 67,  1, 73, 71, 57, 22,  1, 66, 67,  1, 73,
        71, 57,  1, 53, 72,  1, 53, 64, 64, 10,  1, 43, 60, 57,  1, 55, 60, 61,
        64, 56, 70, 57, 66,  1, 75, 67, 66,  5, 72,  1, 64, 57, 72,  1, 65, 57,
         1, 71, 72, 67, 68,  1, 72, 57, 64, 64, 61, 66, 59,  1, 72, 53, 64, 57,
        71,  0, 67, 58,  1, 72, 60, 57,  1, 35, 53, 66, 56,  1, 67, 58,  1, 38,
        78, 10,  1, 32,  1, 63, 66, 67, 75,  1])


In [5]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  # print(ix)
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  x, y = x.to(device), y.to(device)
  return x, y

x, y = get_batch('train')
print('inputs: ')
# print(x.shape)
print(x)
print('targets: ')
print(y)

inputs: 
tensor([[72, 60, 67, 71, 57,  1, 68, 70],
        [ 1, 75, 53, 71,  1, 61, 66,  1],
        [77, 10,  1,  3, 27, 67,  1, 66],
        [ 1, 72, 75, 67,  1, 68, 53, 72]])
targets: 
tensor([[60, 67, 71, 57,  1, 68, 70, 57],
        [75, 53, 71,  1, 61, 66,  1, 70],
        [10,  1,  3, 27, 67,  1, 66, 67],
        [72, 75, 67,  1, 68, 53, 72, 60]])


In [6]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [7]:
class BigramLanguageModel(nn.Module):
  def __init__(self, vocabulary_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocabulary_size, vocabulary_size)

  def forward(self, index, targets=None):
    logits = self.token_embedding_table(index)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, index, max_new_tokens):
    # index is (B,T) array of indices in the current context
    for _ in range(max_new_tokens):
      # get the predictions
      logits, loss = self.forward(index)
      # focus only on the last time step
      logits = logits[:, -1, :] # becomes (B, C)
      # apply softmax to get probabilities
      probs = F.softmax(logits, dim=-1) # (B, C)
      # sample from the distribution
      index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
      # append sampled index to the running sequence
      index = torch.cat((index, index_next), dim=1) # (B, T+1)
    return index

model = BigramLanguageModel(vocabulary_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


:Ul]Gw T0[?1N.Q'o]rrM)S8jIx(M_CKNzv:ch6"co[bI-t9C,Kj(6f8]LA8kl?zO&_45E5nuW7KYp,PCQpH﻿k(WDn3F(F2?R_; zl'Ny ZRYt_Dg&?'?-vQGypNs;?1A
_WET
9fFiV8riv7xK"Zs6(E?zKOXnM.9NA"D2KP2K6AMrco;;;Cz?jwgHMaO)xwM45H'E"2q﻿kQho;8k62fsvhWBv!LcWms(h?-ggoaOM3weU6"u46"1NTH6CkHmNRL&,P?LZq8r6zrhp,GU"1zl p1rb490_96J:)6p299]lIeg
bpZeMXi;8&WVqfYVFVmMT&1r0Z.ZSi9TyuF3
? pTEv OL8&6
"y﻿Bu:_B!uVQ,uFkSS-i69jnGhdv!b]49,IEVI
ca.[y&y2ywghiv50DJ?YuE?)61-y8rw]fPqqH?VFdit!B.RbSvCK:tDoSmKHOT:k_9J'9aj
gKi.xrU;?smH 50ujy'&?bYAr)pcMsm]L0Z8


In [26]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.4f}, val loss: {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 2.6379, val loss: 2.6579
step: 250, train loss: 2.5932, val loss: 2.6441
step: 500, train loss: 2.6278, val loss: 2.6590
step: 750, train loss: 2.5850, val loss: 2.6566
2.4833736419677734


In [27]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


FKns y7_:-gy lfofothe ald M5[!pant
tlarokeYLDJlilopaie gHes t. oicore rde t ant tss fr g

M3youitenqn."D
y)icE0Luede;Q2KWzXind sed
ind.jW8z6(24Rme. be fp,  terd tit atap,Y(!-gre;?HOz﻿QE-Jse,L]
uim
er burlk[She sur hebe ho fBveml NTR;Kkinll s
t ad

TVS[NTY:"2qSWsaml;D)BqqUar5btth.D pR
g--F)Uwer


[VC2h  ure tcalke andrcsk;8'2bDz5p!M6&ksm﻿g-Hdokld It ado'xThs, y beklld
cicis, se!M
: p  o hend ahine g695Nigg ad  mTets T wagge sups, pfPAey RBYVjinueylite
"A?; se
Anth  ofam
t ang er v5-sq; Beazad tar


-

-

-

torch-examples / playing about

In [26]:
import torch

In [27]:
randint = torch.randint(-100, 100, (6,))
randint

tensor([-40,  82,  68,  89, -72, -68])

Embedding Vectors

In [28]:
import torch
import torch.nn as nn

# initialise an embedding layer
vocab_size = 1000
embedding_dim = 100
embedding = nn.Embedding(vocab_size, embedding_dim)

# create some input indices
input_indices = torch.LongTensor([1, 5, 3, 2])

# apply the embedding layer
embedded_output = embedding(input_indices)

# the output will be a tensor of shape (4, 100), where 4 is the number of inputs
# and 100 is the dimensionality of the embedding vectors
print(embedded_output.shape)

torch.Size([4, 100])


In [29]:


x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print('when input is', context, 'target is', target)

when input is tensor([79]) target is tensor(32)
when input is tensor([79, 32]) target is tensor(72)
when input is tensor([79, 32, 72]) target is tensor(5)
when input is tensor([79, 32, 72,  5]) target is tensor(71)
when input is tensor([79, 32, 72,  5, 71]) target is tensor(1)
when input is tensor([79, 32, 72,  5, 71,  1]) target is tensor(66)
when input is tensor([79, 32, 72,  5, 71,  1, 66]) target is tensor(67)
when input is tensor([79, 32, 72,  5, 71,  1, 66, 67]) target is tensor(1)
